In [1]:
from textblob import TextBlob
from googletrans import Translator
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

In [2]:
# Comentarios ficticios sobre reseñas de un libro
comentarios = [
    "Me encantó este libro, fue inspirador y emocionante de principio a fin.",
    "No pude dejar de leerlo, cada página era más intrigante que la anterior.",
    "El desarrollo de los personajes fue impresionante, realmente sentí que los conocía.",
    "La trama era predecible y el final fue decepcionante.",
    "No conecté con los personajes, me resultaron poco realistas.",
    "Este libro me dejó pensando durante días, realmente me hizo reflexionar sobre la vida.",
    "La escritura era hermosa y poética, cada palabra estaba cuidadosamente elegida.",
    "No recomendaría este libro a nadie, no vale la pena el tiempo de lectura.",
    "No puedo esperar para leer más del autor, definitivamente me ha ganado como fan.",
    "La historia me atrapó desde la primera página, no podía ponerlo en espera.",
    "No puedo entender por qué este libro recibió tantas críticas positivas, no lo encontré interesante en absoluto.",
    "Me sentí completamente absorbido por la narrativa, fue como si estuviera viviendo la historia.",
    "La prosa era confusa y difícil de seguir, tuve que volver a leer varias veces para entender lo que estaba pasando.",
    "Fue una lectura ligera y entretenida, perfecta para relajarse en un día lluvioso.",
    "La trama era original y refrescante, no había leído nada parecido antes.",
    "Los diálogos eran realistas y convincentes, realmente sentí que estaba escuchando a personas reales hablar.",
    "Este libro superó mis expectativas, definitivamente lo recomendaría a cualquiera que esté buscando una buena lectura.",
    "No puedo expresar lo mucho que disfruté este libro, me hizo reír y llorar en igual medida.",
    "Fue una montaña rusa emocional, nunca sabía qué esperar a continuación.",
    "Los personajes secundarios eran tan interesantes como los protagonistas, cada uno tenía su propia historia convincente."
]

# Crear un objeto Translator
translator = Translator()

# Inicializar el analizador de sentimientos VADER
sid = SentimentIntensityAnalyzer()

# Función para realizar el análisis de sentimientos con TextBlob y devolver un porcentaje de confianza
def analizar_sentimiento_textblob(comentario):
    blob = TextBlob(comentario)
    polaridad = blob.sentiment.polarity
    subjetividad = blob.sentiment.subjectivity
    
    # Calcular la confianza como el valor absoluto de la polaridad + subjetividad
    confianza = abs(polaridad) + subjetividad
    
    if polaridad > 0:
        return "Positivo", confianza
    elif polaridad < 0:
        return "Negativo", confianza
    else:
        return "Neutral", confianza

# Función para realizar el análisis de sentimientos con VADER
def analizar_sentimiento_vader(comentario):
    polaridad = sid.polarity_scores(comentario)['compound']
    if polaridad > 0:
        return "Positivo"
    elif polaridad < 0:
        return "Negativo"
    else:
        return "Neutral"

# Lista para almacenar resultados del análisis de sentimientos con TextBlob
resultados_textblob = []

# Lista para almacenar resultados del análisis de sentimientos con VADER
resultados_vader = []

# Traducir cada comentario al inglés, luego analizar sentimientos con TextBlob y VADER
for comentario in comentarios:
    # Traducir el comentario al inglés
    comentario_ingles = translator.translate(comentario, src='es', dest='en').text
    
    # Analizar sentimientos con TextBlob
    sentimiento_textblob, confianza_textblob = analizar_sentimiento_textblob(comentario_ingles)
    resultados_textblob.append((comentario, sentimiento_textblob, confianza_textblob))
    
    # Analizar sentimientos con VADER
    sentimiento_vader = analizar_sentimiento_vader(comentario_ingles)
    resultados_vader.append((comentario, sentimiento_vader))

# Crear DataFrames para los resultados
df_textblob = pd.DataFrame(resultados_textblob, columns=['Comentario', 'Sentimiento TextBlob', 'Confianza TextBlob'])
df_vader = pd.DataFrame(resultados_vader, columns=['Comentario', 'Sentimiento VADER'])

# Mostrar los resultados
print("Resultados del análisis de sentimientos con TextBlob:")
print(df_textblob)
print("\nResultados del análisis de sentimientos con VADER:")
print(df_vader)


Resultados del análisis de sentimientos con TextBlob:
                                           Comentario Sentimiento TextBlob  \
0   Me encantó este libro, fue inspirador y emocio...             Positivo   
1   No pude dejar de leerlo, cada página era más i...             Positivo   
2   El desarrollo de los personajes fue impresiona...             Positivo   
3   La trama era predecible y el final fue decepci...             Negativo   
4   No conecté con los personajes, me resultaron p...             Negativo   
5   Este libro me dejó pensando durante días, real...             Positivo   
6   La escritura era hermosa y poética, cada palab...             Positivo   
7   No recomendaría este libro a nadie, no vale la...             Negativo   
8   No puedo esperar para leer más del autor, defi...             Positivo   
9   La historia me atrapó desde la primera página,...             Positivo   
10  No puedo entender por qué este libro recibió t...             Positivo   
11  Me sen

## Importar a CSV 

In [3]:
import csv

In [4]:
# Guardar el DataFrame en un archivo CSV en una ruta específica
ruta_archivo = '/Users/andersongarcesmolina/Documents/data-python/anlisis-sentimientos/csv-sentimientos/df_datos.csv'
df_textblob.to_csv(ruta_archivo, index=False)


In [5]:
# Guardar el DataFrame en un archivo CSV
df_textblob.to_csv('df_textblob.csv', index=False)

# Analizar palabras claves

In [11]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:

# Función para preprocesar los comentarios
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Tokenizar palabras
    tokens = word_tokenize(text)
    # Eliminar palabras vacías
    stop_words = set(stopwords.words('spanish'))  # Cambiar 'spanish' por el idioma correspondiente
    tokens = [word for word in tokens if word not in stop_words]
    # Unir tokens en una cadena de texto nuevamente
    text = ' '.join(tokens)
    return text

# Preprocesar los comentarios
comentarios_procesados = [preprocess_text(comment) for comment in comentarios]

# Extracción de características utilizando TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Seleccionar las 1000 características más importantes
tfidf_matrix = tfidf_vectorizer.fit_transform(comentarios_procesados)

# Obtener las características seleccionadas (palabras clave)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Calcular la importancia de cada palabra clave
importancia_palabras = tfidf_matrix.sum(axis=0)

# Crear un DataFrame con las palabras clave y su importancia
df_palabras_clave = pd.DataFrame({'Palabra': feature_names, 'Importancia': importancia_palabras.tolist()[0]})

# Ordenar las palabras clave por importancia
df_palabras_clave = df_palabras_clave.sort_values(by='Importancia', ascending=False)

# Mostrar las palabras clave más importantes
print("Palabras clave más importantes:")
df_palabras_clave.head(10)


Palabras clave más importantes:


,Palabra,Importancia
59,libro,1.433169
75,personajes,1.071253
104,sentí,0.955733
92,realmente,0.942977
55,lectura,0.925935
47,historia,0.912972
7,cada,0.903975
88,puedo,0.876834
110,trama,0.854917
91,realistas,0.786767


In [33]:
import nltk
from nltk.corpus import stopwords
from collections import Counter

# Descargar los recursos necesarios de nltk (descomentar si es la primera vez que se ejecuta)
# nltk.download('punkt')
# nltk.download('stopwords')

# Definir los comentarios
comentarios = [
    "Me encantó este libro, fue inspirador y emocionante de principio a fin.",
    "No pude dejar de leerlo, cada página era más intrigante que la anterior.",
    "El desarrollo de los personajes fue impresionante, realmente sentí que los conocía.",
    "La trama era predecible y el final fue decepcionante.",
    "No conecté con los personajes, me resultaron poco realistas.",
    "Este libro me dejó pensando durante días, realmente me hizo reflexionar sobre la vida.",
    "La escritura era hermosa y poética, cada palabra estaba cuidadosamente elegida.",
    "No recomendaría este libro a nadie, no vale la pena el tiempo de lectura.",
    "No puedo esperar para leer más del autor, definitivamente me ha ganado como fan.",
    "La historia me atrapó desde la primera página, no podía ponerlo en espera.",
    "No puedo entender por qué este libro recibió tantas críticas positivas, no lo encontré interesante en absoluto.",
    "Me sentí completamente absorbido por la narrativa, fue como si estuviera viviendo la historia.",
    "La prosa era confusa y difícil de seguir, tuve que volver a leer varias veces para entender lo que estaba pasando.",
    "Fue una lectura ligera y entretenida, perfecta para relajarse en un día lluvioso.",
    "La trama era original y refrescante, no había leído nada parecido antes.",
    "Los diálogos eran realistas y convincentes, realmente sentí que estaba escuchando a personas reales hablar.",
    "Este libro superó mis expectativas, definitivamente lo recomendaría a cualquiera que esté buscando una buena lectura.",
    "No puedo expresar lo mucho que disfruté este libro, me hizo reír y llorar en igual medida.",
    "Fue una montaña rusa emocional, nunca sabía qué esperar a continuación.",
    "Los personajes secundarios eran tan interesantes como los protagonistas, cada uno tenía su propia historia convincente."
]

# Concatenar todos los comentarios en un solo string
comentarios_concatenados = ' '.join(comentarios)

# Tokenizar el texto en palabras
palabras = nltk.word_tokenize(comentarios_concatenados)

# Convertir las palabras a minúsculas
palabras = [word.lower() for word in palabras]

# Eliminar las palabras de parada (stopwords)
stop_words = set(stopwords.words('spanish'))
palabras_filtradas = [word for word in palabras if word.isalnum() and word not in stop_words]

# Calcular la frecuencia de cada palabra
frecuencia_palabras = Counter(palabras_filtradas)

# Imprimir las 10 palabras más frecuentes y su frecuencia
print("Palabras más frecuentes y su frecuencia:")
for palabra, frecuencia in frecuencia_palabras.most_common(10):
    print(f"{palabra}: {frecuencia}")


Palabras más frecuentes y su frecuencia:
libro: 6
cada: 3
personajes: 3
realmente: 3
sentí: 3
lectura: 3
puedo: 3
historia: 3
página: 2
trama: 2


In [12]:
df_textblob.head()

,Comentario,Sentimiento TextBlob,Confianza TextBlob
0,"Me encantó este libro, fue inspirador y emocio...",Positivo,1.366667
1,"No pude dejar de leerlo, cada página era más i...",Positivo,0.566667
2,El desarrollo de los personajes fue impresiona...,Positivo,1.200000
3,La trama era predecible y el final fue decepci...,Negativo,1.000000
4,"No conecté con los personajes, me resultaron p...",Negativo,1.500000


### Información 
Es posible que el resultado del análisis de sentimientos muestre una polaridad y subjetividad de 0.0 si el texto analizado es neutral y objetivo. Aquí tienes una explicación de los valores que devuelve el método `sentiment` de TextBlob:

- Polaridad: Es un valor numérico que indica la positividad o negatividad del texto. Un valor de 0.0 indica neutralidad, valores mayores que 0.0 indican positividad y valores menores que 0.0 indican negatividad.
- Subjetividad: Es un valor numérico que indica la subjetividad del texto, es decir, si el texto es objetivo o subjetivo. Un valor de 0.0 indica objetividad (hechos objetivos) y un valor de 1.0 indica subjetividad total (opiniones personales).

En tu caso, al obtener una polaridad y subjetividad de 0.0, significa que el texto es neutro y objetivo, no contiene opiniones o emociones claramente positivas o negativas. Es importante recordar que estos resultados son solo una representación del análisis de sentimientos y pueden variar dependiendo del contexto y la complejidad del texto analizado. Si consideras que el texto debería tener una polaridad y subjetividad diferentes, podrías revisar el texto nuevamente o utilizar otras técnicas de análisis de sentimientos para obtener resultados más precisos.

En el contexto del análisis de sentimientos con TextBlob, la "Confianza TextBlob" representa una medida de cuán seguro está el modelo en la clasificación del sentimiento del comentario dado. 

Para cada comentario, TextBlob calcula una puntuación de polaridad y subjetividad. La polaridad representa la medida en que el texto expresa sentimientos positivos o negativos, mientras que la subjetividad representa la medida en que el texto es subjetivo u objetivo. La "Confianza TextBlob" se calcula sumando el valor absoluto de la polaridad y la subjetividad. 

En tus resultados, vemos que los valores de "Confianza TextBlob" son números positivos. Esto indica que TextBlob está bastante seguro en su clasificación de sentimiento para esos comentarios en particular. Cuanto mayor sea el valor de "Confianza TextBlob", mayor será la confianza del modelo en su clasificación del sentimiento del comentario. En general, un valor alto de confianza sugiere que el modelo está más seguro en su predicción. Sin embargo, es importante tener en cuenta que la interpretación exacta de estos valores puede variar según el contexto y la aplicación específica.

In [28]:
total_sentimientos = df_textblob['Sentimiento TextBlob'].value_counts()/df_textblob.shape[0]
total_sentimientos

Sentimiento TextBlob
Positivo    0.75
Negativo    0.20
Neutral     0.05
Name: count, dtype: float64

In [5]:
df_vader.head()

,Comentario,Sentimiento VADER
0,"Me encantó este libro, fue inspirador y emocio...",Positivo
1,"No pude dejar de leerlo, cada página era más i...",Positivo
2,El desarrollo de los personajes fue impresiona...,Positivo
3,La trama era predecible y el final fue decepci...,Negativo
4,"No conecté con los personajes, me resultaron p...",Neutral


In [13]:
# Guardar el DataFrame en un archivo CSV
df_vader.to_csv('df_vader.csv', index=False)

In [36]:
# Ruta completa de la carpeta donde deseas guardar el archivo CSV
ruta_carpeta = '/Users/andersongarcesmolina/Documents/Data Analysis python/anlisis-sentimientos/csv-sentimientos/'

# Nombre del archivo CSV
nombre_archivo = 'df_vader.csv'

# Ruta completa del archivo CSV
ruta_completa = ruta_carpeta + nombre_archivo

# Guardar el DataFrame en un archivo CSV en la carpeta especificada
df_vader.to_csv(ruta_completa, index=False)


# Dos palabras con mayor frecuencia que estan juntas

In [17]:
from collections import Counter
import re


In [18]:

# Convertir los comentarios a texto único
texto = ' '.join(comentarios)

# Tokenizar el texto en palabras
palabras = re.findall(r'\b\w+\b', texto.lower())

# Contar la frecuencia de las palabras consecutivas
pares_palabras = [(palabras[i], palabras[i+1]) for i in range(len(palabras)-1)]
frecuencia_pares = Counter(pares_palabras)

# Encontrar las dos palabras más comunes
top_2_pares = frecuencia_pares.most_common(2)

print("Las dos palabras más comunes que están juntas son:")
for par, frecuencia in top_2_pares:
    print(par, ":", frecuencia, "veces.")


Las dos palabras más comunes que están juntas son:
('este', 'libro') : 6 veces.
('los', 'personajes') : 3 veces.


In [25]:
# Convertir los comentarios a texto único
texto = ' '.join(comentarios)

# Tokenizar el texto en palabras
palabras = re.findall(r'\b\w+\b', texto.lower())

# Filtrar comentarios negativos
comentarios_negativos = [comentario.lower() for comentario in comentarios if "Negativo" in analizar_sentimiento_textblob(comentario)]

# Convertir los comentarios negativos a texto único
texto_negativo = ' '.join(comentarios_negativos)

# Tokenizar el texto negativo en palabras
palabras_negativas = re.findall(r'\b\w+\b', texto_negativo)

# Contar la frecuencia de las palabras consecutivas
tripletas_palabras_negativas = [(palabras_negativas[i], palabras_negativas[i+1], palabras_negativas[i+2]) for i in range(len(palabras_negativas)-2)]
frecuencia_tripletas_negativas = Counter(tripletas_palabras_negativas)

# Encontrar las tres palabras más comunes
top_3_tripletas_negativas = frecuencia_tripletas_negativas.most_common(3)

if top_3_tripletas_negativas:
    print("Las tres palabras más comunes que están juntas en comentarios negativos son:")
    for trio, frecuencia in top_3_tripletas_negativas:
        print(trio, ":", frecuencia, "veces.")
else:
    print("No se encontraron suficientes tríos de palabras en los comentarios negativos.")


No se encontraron suficientes tríos de palabras en los comentarios negativos.


In [27]:
# Filtrar comentarios positivos
comentarios_positivos = [comentario.lower() for comentario in comentarios if "Positivo" in analizar_sentimiento_textblob(comentario)]

# Convertir los comentarios positivos a texto único
texto_positivo = ' '.join(comentarios_positivos)

# Tokenizar el texto positivo en palabras
palabras_positivas = re.findall(r'\b\w+\b', texto_positivo)

# Contar la frecuencia de las palabras consecutivas
tripletas_palabras_positivas = [(palabras_positivas[i], palabras_positivas[i+1], palabras_positivas[i+2]) for i in range(len(palabras_positivas)-2)]
frecuencia_tripletas_positivas = Counter(tripletas_palabras_positivas)

# Encontrar las tres palabras más comunes
top_3_tripletas_positivas = frecuencia_tripletas_positivas.most_common(3)

if top_3_tripletas_positivas:
    print("Las tres palabras más comunes que están juntas en comentarios positivos son:")
    for trio, frecuencia in top_3_tripletas_positivas:
        print(trio, ":", frecuencia, "veces.")
else:
    print("No se encontraron suficientes tríos de palabras en los comentarios positivos.")


Las tres palabras más comunes que están juntas en comentarios positivos son:
('la', 'trama', 'era') : 1 veces.
('trama', 'era', 'original') : 1 veces.
('era', 'original', 'y') : 1 veces.
